In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import numpy as np
import pandas as pd

from biobank_olink.dataset import load_datasets

ol_df, cov_df = load_datasets(cols_na_th=0, rows_na_th=0)

In [16]:
# W danych dodałem zmienną HTNgroup gdzie 0 to normotensja, 1 to nieleczone HTN (SBP>=140 lub DBP>=90) 2
# to "zażywanie leków antyHTN"
cov_df = cov_df[cov_df.HTNgroup < 2]
cov_df = cov_df.sort_values(by='SBP')
cov_df.head(5)

,Sex,age,BMI,Smokinstatus,Alcoholintakefrequency,HTNgroup,fastingtime,SBP,DBP,PP
eid,,,,,,,,,,
1100542,0.0,56.0,25.3302,0.0,2.0,0.0,3.0,76.5,47.0,29.5
2071806,1.0,61.0,32.7542,1.0,4.0,0.0,3.0,84.5,53.0,31.5
4670270,0.0,41.0,21.7004,0.0,4.0,0.0,4.0,84.5,62.0,22.5
3662467,0.0,55.0,20.3475,0.0,3.0,0.0,6.0,85.0,59.0,26.0
5301061,1.0,43.0,21.6512,1.0,6.0,0.0,2.0,85.5,55.5,30.0


In [17]:
ol_df_corr = ol_df.corr()
mask = np.triu(np.ones(ol_df_corr.shape), k=1).astype(bool)
high_corr = ol_df_corr.where(mask)
cols_to_remove = [column for column in high_corr.columns if any(high_corr[column] > 0.9)]
ol_df.drop(columns=cols_to_remove, inplace=True)

In [18]:
threshold = 0.35
lower_bound, upper_bound = cov_df.SBP.quantile([threshold, 1 - threshold]).values
low_cov_df = cov_df[cov_df.SBP < lower_bound]
high_cov_df = cov_df[upper_bound < cov_df.SBP]
pd.concat([low_cov_df.SBP.describe(), high_cov_df.SBP.describe()], axis=1, keys=["low", "high"])

,low,high
count,10679.000000,10721.000000
mean,117.774230,156.075972
std,7.386252,12.429014
min,76.500000,142.000000
25%,113.000000,146.500000
50%,119.500000,153.000000
75%,124.000000,162.000000
max,127.500000,246.000000


In [19]:
correction_df = pd.concat([low_cov_df, high_cov_df])
correction_cols = ["Sex", "age", "BMI"]
correction_df = correction_df[correction_cols]
correction_df = (correction_df - correction_df.mean()) / correction_df.std()
correction_df.head()

,Sex,age,BMI
eid,,,
1100542,-0.837467,-0.017329,-0.306785
2071806,1.194021,0.593013,1.366313
4670270,-0.837467,-1.848354,-1.124809
3662467,-0.837467,-0.139397,-1.429703
5301061,1.194021,-1.604218,-1.135897


In [20]:
from scipy.spatial.distance import pdist, squareform

similarities = squareform(pdist(correction_df))
np.fill_diagonal(similarities, np.inf)
similarities_df = pd.DataFrame(similarities, index=correction_df.index, columns=correction_df.index)
similarities_sub_df = similarities_df.loc[low_cov_df.index, high_cov_df.index]

paired_up_df = similarities_sub_df.idxmin().to_frame("p2_id")
paired_up_df["dist"] = similarities_df.min()
paired_up_df2 = similarities_sub_df.T.idxmin().to_frame("p2_id")
paired_up_df2["dist"] = similarities_df.T.min()
paired_up_df = pd.concat([paired_up_df, paired_up_df2])
paired_up_df.sort_values(by="dist", inplace=True)

del similarities_df, similarities
paired_up_df.head()

,p2_id,dist
eid,,
3636713,4051783,0.0
2389551,2986916,0.0
1528084,5331657,0.0
2116722,4731896,0.0
2263355,5522598,0.0


In [21]:
p1, p2, _ = paired_up_df.reset_index().iloc[-1].values
cov_df.loc[[p1, p2]]

,Sex,age,BMI,Smokinstatus,Alcoholintakefrequency,HTNgroup,fastingtime,SBP,DBP,PP
eid,,,,,,,,,,
5519244,1.0,65.0,50.9975,1.0,3.0,1.0,4.0,150.0,90.0,60.0
1939696,1.0,55.0,48.3703,0.0,3.0,0.0,12.0,127.0,86.5,40.5


In [22]:
chosen = set()
for p1_idx, (p2_idx, _) in paired_up_df.iterrows():
    if p1_idx in chosen or p2_idx in chosen:
        continue
    chosen.add(p1_idx)
    chosen.add(p2_idx)

chosen_cov_df = cov_df.loc[list(chosen)]
low_cov_df = chosen_cov_df[chosen_cov_df.SBP < lower_bound]
high_cov_df = chosen_cov_df[upper_bound < chosen_cov_df.SBP]
pd.concat([low_cov_df.SBP.describe(), high_cov_df.SBP.describe()], axis=1, keys=["low", "high"])

,low,high
count,5133.000000,5133.000000
mean,118.922950,155.348821
std,6.843063,12.172580
min,76.500000,142.000000
25%,115.000000,146.000000
50%,120.500000,152.000000
75%,124.500000,161.000000
max,127.500000,246.000000


In [23]:
pd.concat([low_cov_df[correction_cols].describe(), high_cov_df[correction_cols].describe()], axis=1,
          keys=["low", "high"])

low                                   high               \
               Sex          age          BMI          Sex          age   
count  5133.000000  5133.000000  5133.000000  5133.000000  5133.000000   
mean      0.431522    56.676018    26.796728     0.431522    56.678745   
std       0.495337     7.627332     4.289300     0.495337     7.626911   
min       0.000000    40.000000    15.353500     0.000000    40.000000   
25%       0.000000    51.000000    23.843700     0.000000    51.000000   
50%       0.000000    58.000000    26.228400     0.000000    58.000000   
75%       1.000000    63.000000    29.039200     1.000000    63.000000   
max       1.000000    70.000000    55.134100     1.000000    70.000000   

                    
               BMI  
count  5133.000000  
mean     26.798973  
std       4.289026  
min      15.657300  
25%      23.857200  
50%      26.225200  
75%      29.054200  
max      56.501900

## Classification

In [26]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

x = ol_df.loc[chosen_cov_df.index]
y = chosen_cov_df.index.isin(high_cov_df.index)

est = XGBClassifier(tree_method="gpu_hist", random_state=42, n_estimators=1000)
scores = cross_val_score(est, x, y, scoring="accuracy", cv=5, n_jobs=5)
scores, scores.mean()

(array([0.6976631 , 0.71115441, 0.71358987, 0.70725767, 0.7116415 ]),
 0.7082613090018122)